In [1]:
import folium
from folium.plugins import MarkerCluster
import os
import json
from tqdm import tqdm
import branca.colormap as cm

# database imports
from sqlalchemy import create_engine, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.exc import NoResultFound
from geoalchemy2 import Geometry, WKTElement
from geoalchemy2.shape import from_shape, to_shape
from shapely.geometry import shape, mapping
from shapely.wkb import loads

# data models
from database import Base, Feature, FeatureSet, Style, Colormap

In [2]:
# connect to PostGIS
password = "rescuemate" # TODO: change this later to something proper

DATABASE_URL = "postgresql+psycopg2://postgres:" + password + "@localhost:5432/postgres"
engine = create_engine(DATABASE_URL, echo=False)

print("1")

# Start a session
Session = sessionmaker(bind=engine)
session = Session()

print("2")

# activate postGIS if its not already enabled
session.execute(text("CREATE EXTENSION IF NOT EXISTS postgis"))

print("3")

# add the data models to the database
Base.metadata.create_all(engine)

print("Done!")

1
2
3
Done!


In [16]:
# drop all tables (useful reset, but be careful!)
# Base.metadata.drop_all(engine)

In [17]:
# if necessary, rollback
# session.rollback()

In [3]:
# Create some dummy values
# Insert a new Style
# style = Style(
#     name="Sample Style",
#     color="red",
#     fill_color="red",
#     icon_prefix="fa",
#     icon_name="circle",
#     line_weight=3,
#     popup_properties={
#         "Name": "name",
#         "Description": "description"
#     }
# )

# # Insert a new FeatureSet
# feature_set = FeatureSet(
#     name="Sample FeatureSet",
#     style=style   # link the style to the feature set
# )

# # Insert a new Feature
# feature = Feature(
#     feature_set=feature_set, # link the feature to the feature set
#     properties={
#         "name": "Sample Feature",
#         "description": "This Sample Feature™ has a single point as its geometry."
#     },
#     geometry_type="POINT",
#     geometry=from_shape(shape({
#     "type": "Point",
#     "coordinates": [0, 0]
#     }))
# )

# # add the objects to the session
# session.add(style)
# session.add(feature_set)
# session.add(feature)
# session.commit()

In [4]:
# query all values in all FeatureSets
feature_sets = session.query(FeatureSet).all()
for feature_set in feature_sets:
    print(feature_set.name, feature_set.style.name)

Sample FeatureSet Sample Style


# Handling data in PostGIS

### Inserting
**Single Record**:
```python
location1 = Location(name="Point A", coordinate="POINT(0 0)")
session.add(location1)
session.commit()
```

**Multiple Records**:
```python
locations_list = [
    Location(name="Point B", coordinate="POINT(1 1)"),
    Location(name="Point C", coordinate="POINT(2 2)")
]
session.add_all(locations_list)
session.commit()
```

### Querying
**Query all**:
```python
locations = session.query(Location).all()
for location in locations:
    print(location.name, location.coordinate)
```

**Query with filter**:
```python
point_b = session.query(Location).filter_by(name="Point B").first()
print(point_b.id, point_b.name, point_b.geom)
```

### Updating
```python
point_b.name = "Point B Updated"
session.commit()
```

### Deleting
**Single record**:
```python
session.delete(point_b)
session.commit()
```

**All locations**:
```python
session.query(Location).delete()
session.commit()
```

In [3]:
# gather all geojson files
# geojson files are located in their respective folders under data
# files are then saved in a dictionary with the folder name as key

files = {}

for root, dirs, filenames in os.walk('data'):
    for f in filenames:

        # get the folder name after data\
        # i.e. emobility_json or hafengebiets_json
        folder = root.split('\\')[1]

        # create a new list if the folder name is not in the dictionary
        if folder not in files:
            files[folder] = []
        
        # append the file name to the list
        files[folder].append(f)

In [4]:
files

{'bedrohte_gruppen_json': ['app_perspektive_wohnen_bestehend_EPSG_25832.json',
  'app_perspektive_wohnen_bestehend_EPSG_4326.json',
  'de_hh_up_nicht_staatliche_schulen_EPSG_25832.json',
  'de_hh_up_nicht_staatliche_schulen_EPSG_4326.json',
  'de_hh_up_staatliche_schulen_EPSG_25832.json',
  'de_hh_up_staatliche_schulen_EPSG_4326.json',
  'de_hh_up_vollstationaere_pflegeeinrichtungen_EPSG_25832.json',
  'de_hh_up_vollstationaere_pflegeeinrichtungen_EPSG_4326.json',
  'settings.json'],
 'deichgrundgrenze_json': ['de_hh_up_deichgrundgrenze_EPSG_25832.json',
  'de_hh_up_deichgrundgrenze_EPSG_4326.json',
  'settings.json'],
 'emobility_json': ['app_stromnetz_emobility_EPSG_25832.json',
  'app_stromnetz_emobility_EPSG_4326.json',
  'settings.json'],
 'feuerwehrstandorte_json': ['de_hh_up_berufsfeuerwehr_EPSG_25832.json',
  'de_hh_up_berufsfeuerwehr_EPSG_4326.json',
  'de_hh_up_freiwillige_feuerwehr_EPSG_25832.json',
  'de_hh_up_freiwillige_feuerwehr_EPSG_4326.json',
  'settings.json'],
 'gew

In [5]:
# takes in a path to a file and returns the json object
def load_json(json_path, encoding='utf-8'):
    json_data = {}
    with open(json_path, 'r', encoding=encoding) as settings_file:
        json_data = json.load(settings_file)
    return json_data

In [22]:
# session.rollback()

In [8]:
def transform_geojson_to_db(files, base_path='data'):
    """
    Transforms geojson files into database entries.
    Takes in files as a dictionary with the folder name as key and a list of files as value.
    These files are then processed and saved into the database.
    Each folder name should contain a settings.json file.
    """
    
    for category, file_list in tqdm(files.items()):

        # get the settings file for the category
        settings_path = os.path.join(base_path, category, 'settings.json')
        
        if os.path.exists(settings_path):
            category_settings = load_json(settings_path)
            
            # process each file in the category
            for file_name, file_settings in category_settings['files'].items():

                # only process files with the "standard" EPSG_4326 projection
                if "EPSG_4326" in file_name:
                    geojson_path = os.path.join(base_path, category, file_name)
                    geojson_data = load_json(geojson_path)

                    # Get the respective FeatureSet
                    feature_set = session.query(FeatureSet).filter_by(name=file_settings['name']).first()

                    # Create a new FeatureSet if it doesn't exist yet
                    if not feature_set:

                        # check for a colormap
                        colormap = None

                        if 'colormap' in file_settings:
                            colormap = Colormap(
                                property=file_settings['colormap']['property'],
                                min_color=file_settings['colormap']['colors'][0],
                                max_color=file_settings['colormap']['colors'][1],
                                min_value=file_settings['colormap']['vmin'],
                                max_value=file_settings['colormap']['vmax']
                            )
                            session.add(colormap)
                            session.commit()

                        # create a new style
                        # takes the style from the first file of settings.json
                        style = Style(
                            name=category_settings['display_name'],
                            popup_properties=file_settings.get('popup_properties', {}),
                            color=file_settings.get('fill_color', 'blue'),
                            fill_color=file_settings.get('fill_color', 'black'),
                            icon_prefix=file_settings.get('icon-prefix', 'fa'),
                            icon_name=file_settings.get('icon', 'circle'),
                            line_weight=file_settings.get('line_weight', 1.0),
                            colormap=colormap
                        )
                        session.add(style)
                        session.commit()

                        # finally, create the feature set
                        feature_set = FeatureSet(
                            name=file_settings['name'],
                            style=style
                        )
                        session.add(feature_set)
                        session.commit()

                    # Process each feature
                    for feature in geojson_data['features']:

                        # Skip features without a geometry
                        if feature['geometry'] is None:
                            continue

                        # Convert GeoJSON geometry to a Shapely geometry
                        shapely_geom = shape(feature['geometry'])

                        # Use Shapely geometry with `geoalchemy2`
                        geometry_type = shapely_geom.geom_type
                        wkt_geometry = shapely_geom.wkt
                        srid = 4326
                        geometry_element = WKTElement(wkt_geometry, srid)

                        # Get the properties of the feature
                        properties = feature.get('properties', {})

                        # finally, create the feature
                        feature = Feature(
                            feature_set=feature_set,
                            properties=properties,
                            geometry_type=geometry_type,
                            geometry=geometry_element
                        )

                        session.add(feature)

                    session.commit()

In [9]:
# this function takes all files and saves them in the database
# already done, so we dont need to run it again
# transform_geojson_to_db(files)

In [6]:
# function to get all features from a feature set
def get_features(feature_set_id):
    return session.query(Feature).filter_by(feature_set_id=feature_set_id).all()

In [7]:
# print all feature set names and their ids
for feature_set in session.query(FeatureSet).all():
    print(feature_set.id, feature_set.name)

1 Sample FeatureSet
2 Flüchtlingsunterkunft
3 Nicht-staatliche Schule
4 Staatliche Schule
5 Pflegeeinrichtung
6 Deichgrundgrenze
7 Ladestation
8 Berufsfeuerwehr
9 Freiwillige Feuerwehr
10 Gewässer Einzugsgebiet
11 Hafengebietsgrenze
12 Hauptdeichlinie
13 Kita
14 Krankenhaus
15 Kommissariatsgebiet
16 Polizei
17 Straße


## Possible colors for Folium:

```{'lightgray', 'gray', 'beige', 'darkgreen', 'black', 'red', 'darkblue', 'darkred', 'lightblue', 'green', 'white', 'cadetblue', 'pink', 'lightred', 'lightgreen', 'darkpurple', 'purple', 'blue', 'orange'}```

Or just use a hex color code, e.g. ```'#FF0000'```

In [4]:
m = folium.Map(location=(53.55, 9.99), zoom_start=12)

In [6]:
# GeoJson and Marker objecs will be styled like this
def create_style_function(style):
    def style_function(feature):

        # Default color from style settings
        fillColor = style.fill_color
        color = style.color
        line_weight = style.line_weight

        # If there's a colormap, compute the color based on feature value
        # [You will need to implement colormap logic if required, based on your style model]

        # If there's a colormap, compute the color based on feature value
        if style.colormap:

            colormap = style.colormap

            min_color, max_color = colormap.min_color, colormap.max_color
            min_value, max_value = colormap.min_value, colormap.max_value
            colormap_property = colormap.property

            # get the value from the features property
            # if it doesnt exist, use the minimum value
            feature_properties = feature.get('properties', {})
            property_value = feature_properties.get(colormap_property, colormap.min_value)

            # compute the color
            linear_colormap = cm.LinearColormap(
                colors=[min_color, max_color],
                vmin=min_value,
                vmax=max_value,
            )

            # set the color
            fillColor = linear_colormap(property_value)[:7]
            color = fillColor

        return {
            'fillColor': fillColor,
            'color': color,
            'weight': line_weight
        }
    return style_function

In [ ]:
# get all feature_sets
# feature_sets = session.query(FeatureSet).all()
feature_sets = session.query(FeatureSet).filter(FeatureSet.id != 17).all()  # exclude Straße, very big dataset

for feature_set in tqdm(feature_sets):
    
    # Create a FeatureGroup for this set of features
    feature_group = folium.FeatureGroup(name=feature_set.name, show=False)

    # get the style of this feature
    style = feature_set.style

    # only display if there is a style for this feature
    if style == None:
        continue

    # get the popup properties of this feature
    popup_properties = style.popup_properties
    
    for feature in feature_set.features:

        # get the style function
        style_function = create_style_function(style)
        
        # For popup
        popup_content = f"<b>{feature_set.name}</b><br>"

        for property in popup_properties:
            current_property = popup_properties[property]
            value = feature.properties.get(current_property, '')
            popup_content += f"<b>{property}</b>: {value}<br>"
        
        geometry_type = feature.geometry_type

        # Create a Marker or a GeoJSON object depending on the geometry type
        if geometry_type.upper() == 'POINT':

            # Query the Point's geometry
            x, y = session.query(func.ST_X(Feature.geometry), func.ST_Y(Feature.geometry)).filter(Feature.id == feature.id).first()

            # get the icon information
            icon_prefix = style.icon_prefix
            icon_name = style.icon_name

            folium.Marker(
                location=[y, x], # Note the switch, as folium expects [lat, lon]
                popup=popup_content,
                icon=folium.Icon(icon=icon_name, prefix=icon_prefix, color=style.color),
                show=False
            ).add_to(feature_group)

        else:
            # Convert WKB to a Shapely geometry
            shape_geometry = loads(bytes(feature.geometry.data))

            # Convert the Shapely geometry to GeoJSON
            geojson_geometry = mapping(shape_geometry)

            # print(style.fill_color, style.color, style.line_weight)

            geojson_feature = {
                'type': 'Feature',
                'properties': feature.properties,
                'geometry': geojson_geometry
            }

            folium.GeoJson(
                geojson_feature, 
                style_function=style_function, 
                tooltip=popup_content,
                show=False
            ).add_to(feature_group)
            
    feature_group.add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
# save the map to an html file
m.save('map.html')